# Librerías

In [1]:
from attentional_cpmp.utils import load_data
from attentional_cpmp.utils import load_data_mongo
from attentional_cpmp.utils import connect_to_server
from attentional_cpmp.model import load_cpmp_model
from attentional_cpmp.model import create_model
from attentional_cpmp.model import reinforcement_training
from cpmp_ml.optimizer import GreedyV1
from cpmp_ml.optimizer import GreedyV2
from cpmp_ml.utils.adapters import AttentionModel
from cpmp_ml.validations import validate_model
from dotenv import load_dotenv
from urllib.parse import quote_plus
import os

# Cargar Data

In [4]:
load_dotenv()

db_host = os.environ.get("DB_HOST")
db_user = os.environ.get("DB_USER")
db_password = os.environ.get("DB_PASSWORD")
db_name = os.environ.get("DB_NAME")

MONGO_URI_MODEL_V2 = f'mongodb://{quote_plus(db_user)}:{quote_plus(db_password)}@{db_host}/?authSource={db_name}'
MONGO_URI_LOCAL = "mongodb://localhost:27017/"

## Local

In [ ]:
data_5x5, labels_5x5 = load_data('Data_CPMP/Data_Model_v2/data_5x5_Multiple.csv')

In [ ]:
data_7x7, labels_7x7 = load_data('data/data_7x7_Multiple.csv')

## MongoDB

### Modelo v2

In [ ]:
client_v2 = connect_to_server(MONGO_URI_MODEL_V2)

base_de_datos = client_v2.data_Model_v2
data_5x5, labels_5x5 = load_data_mongo(base_de_datos.data_5x5)

client_v2.close()

In [ ]:
client_v2 = connect_to_server(MONGO_URI_MODEL_V2)

base_de_datos = client_v2.data_Model_v2
data_7x7, labels_7x7 = load_data_mongo(base_de_datos.data_7x7)

client_v2.close()

### Local

In [5]:
client_local = connect_to_server(MONGO_URI_MODEL_V2)

base_de_datos = client_local['CPMP_With_Attention']
data_Sx7 = load_data_mongo(base_de_datos["Sx7"])

client_local.close()

Conection Success


# Cargar Modelos

In [4]:
model_Sx7 = load_cpmp_model("./models/Sx7/model_Sx7.h5")

# Modelo Sx5

In [5]:
model_Sx5 = create_model(5, 5, 'Adam', num_stacks= 3)

## Entrenamiento

In [5]:
for stack in data_5x5:
    model_Sx5.fit(data_5x5[stack]["States"], data_5x5[stack]["Labels"], batch_size= 32, epochs= 50)

Epoch 1/50
6222/6222 [==============================] - 80s 12ms/step - loss: 0.1439 - mae: 0.0866 - mse: 0.0429
Epoch 2/50
6222/6222 [==============================] - 75s 12ms/step - loss: 0.0968 - mae: 0.0573 - mse: 0.0287
Epoch 3/50
6222/6222 [==============================] - 75s 12ms/step - loss: 0.0877 - mae: 0.0516 - mse: 0.0258
Epoch 4/50
6222/6222 [==============================] - 74s 12ms/step - loss: 0.0822 - mae: 0.0483 - mse: 0.0241
Epoch 5/50
6222/6222 [==============================] - 75s 12ms/step - loss: 0.0778 - mae: 0.0457 - mse: 0.0228
Epoch 6/50
6222/6222 [==============================] - 74s 12ms/step - loss: 0.0744 - mae: 0.0438 - mse: 0.0218
Epoch 7/50
6222/6222 [==============================] - 75s 12ms/step - loss: 0.0720 - mae: 0.0425 - mse: 0.0212
Epoch 8/50
6222/6222 [==============================] - 81s 13ms/step - loss: 0.0703 - mae: 0.0416 - mse: 0.0207
Epoch 9/50
6222/6222 [==============================] - 76s 12ms/step - loss: 0.0690 - mae: 0.04

## Testing

In [6]:
validate_model(model_Sx5, GreedyV1(), AttentionModel(), 5, 5, 15, size_states= 1000, max_steps= 30)

success heuristic (%): 6.0 6.35
mean steps: 6.35
median steps: 6.0
min steps: 1.0
max steps: 10.0



(0.0, 6.0)

In [7]:
validate_model(model_Sx5, GreedyV2(), AttentionModel(), 5, 5, 15, size_states= 1000, max_steps= 30)

success ann model (%): 0.1
mean steps: 0.0
median steps: 0.0
min steps: 0.0
max steps: 0.0

success heuristic (%): 100.0 12.286
mean steps: 12.286
median steps: 12.0
min steps: 0.0
max steps: 27.0



(0.1, 100.0)

In [ ]:
model_Sx5.save("./models/model_Sx5.h5")

## Model Sx7

In [6]:
model_Sx7 = create_model(num_heads= 5, key_dim= 8, H= 7, num_stacks= 7)

c:\Users\matia\Desktop\Workspace\CPMP_With_attention\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Entrenamiento

In [ ]:
for stack in data_Sx7:
    model_Sx7.fit(data_Sx7[stack]["States"], data_Sx7[stack]["Labels"], batch_size= 32, epochs= 10)

In [10]:
model_Sx7.save("./models/Sx7/model_Sx7.h5")

/Users/slinking/Desktop/Workspace/CPMP_With_attention/.venv/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
validate_model(model_Sx7, GreedyV2(), AttentionModel(), 10, 7, 50, 1000, max_steps= 100)

# Reinforcement Learning

In [ ]:
reinforcement_training(model_Sx5, 5, 5, 15, GreedyV2(), AttentionModel(), iter= 2, max_steps= 30, epochs= 10, batch_size= 50)